In [ ]:
import torch
import matplotlib.pyplot as plt
from pathlib import Path

def plot_image(images, rescale_method="clamp", name="temp_image"):
    # Create the 4x4 grid
    fig, axes = plt.subplots(4, 4, figsize=(6, 6))
    axes = axes.flatten()

    for img, ax in zip(images, axes):
        if rescale_method == "tanh":
            img = torch.tanh(img)
        elif rescale_method == "clamp":
            img = torch.clamp(img, -1.0, 1.0)
        elif rescale_method == "none":
            pass
        else:
            raise ValueError("Unsupported rescale method")
        img = (img + 1) / 2
        img = img.permute(1, 2, 0)
        ax.imshow(img)
        ax.axis("off")

    plt.tight_layout()
    Path("./images").mkdir(parents=True, exist_ok=True)
    plt.savefig(f"./images/{name}.png")
    plt.close(fig)

images = torch.load("./images/image_batch.pth", map_location="cpu")
plot_image(images[:16], rescale_method="clamp", name=f"images")

In [ ]:
from vae import vae

import torch
import matplotlib.pyplot as plt

checkpoint = torch.load(f"../trained_models/VAE_model.pth", map_location="cpu")
model = vae.VAE(
    in_channels=checkpoint["in_channels"],
    in_dim=checkpoint["in_dim"],
    latent_dim=checkpoint["latent_dim"],
)
model.load_state_dict(checkpoint["model_state_dict"])
batch_size = 16
samples = model.sample(batch_size)
samples = samples.reshape(batch_size, 1, 28, 28)

plot_image(samples[:16], rescale_method="clamp", name=f"vae_images")

train_loss = checkpoint["train_loss"]
val_loss = checkpoint["val_loss"]
assert len(train_loss) == len(val_loss)
n_epochs = [*range(len(train_loss))]

plt.style.use("default")
plt.plot(n_epochs, train_loss, label="train loss")
plt.plot(n_epochs, val_loss, label="val loss")
plt.xlim(0, 10)
plt.ylim(0, 100)
plt.legend()
plt.show()